In [1]:
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode, create_react_agent
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from IPython.display import Image, display
from typing import Literal, TypedDict, Annotated, List
import operator
import os

print("Successfully imported the neccessaries")

Successfully imported the neccessaries


In [2]:
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("Load your key")

print("API key loaded")

API key loaded


In [4]:
llm =  ChatOpenAI(
    model = "gpt-5-nano",
    temperature = 0,
    api_key = api_key
)
print(f"{llm.model_name} Model has been  defined")

gpt-5-nano Model has been  defined


In [5]:
# creating the hybrid state

class HybridState(TypedDict):
    """"Combined state for plan_execute and Reflection pattern"""

    input: str
    plan: list[str]
    current_step: int
    results: Annotated[List[str], operator.add]
    #reflection
    draft: str
    critique: str
    iterations: int
    final_output: str

MAX_REFLECTION = 2
print("Defined the hybrid state!")



Defined the hybrid state!


In [ ]:
# lets define the nodes
# planner node
def planner(state: HybridState) -> dict:
    """Create a step-by-step plan for the task"""
    prompt = f"""Create a step-by-step plan for the task:

Task: {state['input']}
Return a numbered list of concrete steps. Keep it concise(3-5 steps)."""
    response = llm.invoke([HumanMessage(content=prompt)])
    # paerse the steps
    lines = response.content.split('\n')
    steps = [line.strip() for line in lines if line.strip() and any(char.isdigit() for char in line[:3])]

    print(f"\n Plan has been created:")
    for step in steps:
        print(f"  {step}")
    print()
    
    return {"plan": steps, "current_step": 0, "results": []}

# executor node
def executor(state: HybridState) -> dict:
    """Execute each of the plans"""
    if state["current_step"] >= len(state["plan"]):
        #All steps has been executed
        return {}
    
    current_step = state["plan"][state["current_step"]]

    print(f"Executing step: {current_step}")

